# Hilfsfunktionen
Im folgenden sollen Funktionen implementiert werden, die an verschiedenen Stellen in der Entwicklung des Mühlespiels genutzt werden können. Dazu sind sie so einfach wie möglich gehalten.

`count_player_pieces()` zählt die Steine, die ein anzugebender Spieler gerade auf dem Spielfeld hat.

In [ ]:
def count_player_pieces(state, player):
    return [position for ring in state[1] for position in ring].count(player)

`empty_positions()` gibt eine Liste mit Tupeln zurück. Jedes Tupel beschreibt eine Stelle auf dem Spielfeld, auf der kein Spielstein steht. So erhält man alle leeren Positionen.

In [ ]:
def empty_positions(state):
    return [(ring, cell) for ring in range(3) for cell in range(8) if state[1][ring][cell] == 0]

`neighboring_positions()` liefert für eine gegebene Position auf dem Spielfeld alle benachbarten Felder. Die Position wird dabei in der der Form `(ring, cell)` übergeben. Je nachdem wo sich die übergebene Stelle befindet, hat sie 2, 3 oder 4 Nachbarfelder, die es zu ermitteln gilt.

In [ ]:
def neighboring_positions(position):
    ring, cell = position
    
    left_neighbor  = (ring, (cell - 1) % 8)
    right_neighbor = (ring, (cell + 1) % 8)
        
    positions = [left_neighbor, right_neighbor]
        
    if cell in [1, 3, 5, 7]:
        if ring in [0, 2] :
            positions.append((1, cell))
        else:
            positions.append((ring - 1, cell))
            positions.append((ring + 1, cell))
    return positions

`empty_neighbors()` berechnet ausgehend von einem gewählten Stein alle benachbarten leeren Felder.
Hierfür wird ein Spielstatus, sowie die Position des ausgewählten Steines mit ring und cell übergeben. Zurückgegeben wird eine Liste der möglichen Position, auf die gezogen werden kann.

In [ ]:
def empty_neighbors(state, ring, cell):
    neighboring = set(neighboring_positions((ring, cell)))
    empty = set(empty_positions(state))
    empty_neighbors = neighboring.intersection(empty)
    return list(empty_neighbors)

`player_pieces()` gibt eine Liste mit allen Positionen eines zu übergebenden Spielers zurück.

In [ ]:
def player_pieces(state, player):
    return [(ring, cell) for ring in range(3) for cell in range(8) if state[1][ring][cell] == player]

`opponent()` gibt den Gegner des übergebenen Spielers zurück.

In [ ]:
def opponent(player):
    return 3 - player

`is_movable()` gibt für einen übergebenen Stein an, ob dieser noch verschiebbar ist.

In [ ]:
def is_movable(state, position):
    neighboring = set(neighboring_positions(position))
    for (ring, cell) in neighboring:
        if state[1][ring][cell] == 0:
            return True
    return False

## Mühlen
`find_mills()` gibt alle Mühlen zurück, die ein Spieler auf dem übergebenen Spielbrett hat
Dazu werden zunächst alle Mühlen identifiziert, die auf dem Spielfeld zu finden sind. Hierzu werden in der ersten Schleife alle Mühlen ermittelt, die sich auf den Ringen befinden. Die zweite Schleife erkennt alle Mühlen, die sich vertikal über alle Ringe erstrecken. Mühlen werden dabei als Tupel der Form `((ring 1, cell 1), (ring 2, cell 2), (ring 3, cell 3))` gespeichert.

In [ ]:
def find_mills(board, player):
    mills = set()

    for ring in range (0,3):
        for cell in [0,2,4,6]:
            if(board[ring][cell] == board[ring][cell + 1] == board[ring][(cell + 2) % 8] == player):
                mills.add(((ring, cell), (ring, cell + 1), (ring, (cell + 2) % 8)))
    for cell in [1,3,5,7]:
        if(board[0][cell] == board[1][cell] == board[2][cell] == player):
            mills.add(((0, cell), (1, cell), (2, cell)))            
    return mills

`count_new_mills()` nimmt zwei Mengen von Mühlen und berechnet die Anzahl der Unterschiede zwischen `old_mills` und `new_mills`.

In [ ]:
def count_new_mills(old_mills, new_mills):
    return len(new_mills.difference(old_mills))

`get_opponent_beatable_pieces()` gibt eine Liste mit entfernbaren Steinen des Gegners zurück. Dies ist notwendig, wenn auf Grund des Bildens einer Mühle Steine des Gegners entfernt werden dürfen. Laut den Regeln dürfen keine Steine aus Mühlen entfernt werden, es sei denn, alle Steine des Gegners sind in Mühlen. 
Dazu werden alle Steine sowie alle Mühlsteine des Gegners ermittelt. Die schlagbaren Steine sind die Steine des Spielers ohne dessen Mühlsteine. Ist das Ergebnis jedoch leer, werden alle Steine des Gegners zurückgegeben, da laut den offiziellen Regeln in disem Fall auch Steine aus Mühlen entfernt werden können.

In [ ]:
def get_opponent_beatable_pieces(state, current_player):
    opponent_player = opponent(current_player)

    opponent_mill_pieces = [(ring, cell) for mill in find_mills(state[1], opponent_player) for (ring, cell) in mill]
    opponent_pieces      = player_pieces(state, opponent_player)
    beatable_pieces      = [piece for piece in opponent_pieces if piece not in opponent_mill_pieces ]
    
    if len(beatable_pieces) == 0:
        return list(opponent_pieces)
    return beatable_pieces

`beat_pieces()` gibt alle möglichen Spielfelder zurück, nachdem eine bestimmte Anzahl an gegnerischen Steinen (`count`) entfernt wurde

In [ ]:
def beat_pieces(board, count, player):
    if count <= 0: return [board]
    
    boards = []
    beatable_pieces = get_opponent_beatable_pieces([[0, 0], board], player)
    for ring, cell in beatable_pieces:
        new_board = copy.deepcopy(board)
        new_board[ring][cell] = 0
        boards.extend(beat_pieces(new_board, count - 1, player))
    
    # Duplikate entfernen
    # TODO: eventuell geht das noch schöner
    result = []
    for board in boards:
        if board not in result:
            result.append(board)
    return result

Die Funktion `to_tupel()` wandelt einen übergebenen Spielstatus in Tupelform um.

In [2]:
def to_tupel(state):
    return ((state[0][0], state[0][1]), (
                (state[1][0][0],state[1][0][1],state[1][0][2],state[1][0][3],state[1][0][4],state[1][0][5],state[1][0][6],state[1][0][7]),
                (state[1][1][0],state[1][1][1],state[1][1][2],state[1][1][3],state[1][1][4],state[1][1][5],state[1][1][6],state[1][1][7]),
                (state[1][2][0],state[1][2][1],state[1][2][2],state[1][2][3],state[1][2][4],state[1][2][5],state[1][2][6],state[1][2][7])
            ))

Die Funktion `to_list()` wandelt einen übergebenen Spielstatus in Listenform um.

In [3]:

def to_list(state):
    return [[state[0][0], state[0][1]], [
                [state[1][0][0],state[1][0][1],state[1][0][2],state[1][0][3],state[1][0][4],state[1][0][5],state[1][0][6],state[1][0][7]],
                [state[1][1][0],state[1][1][1],state[1][1][2],state[1][1][3],state[1][1][4],state[1][1][5],state[1][1][6],state[1][1][7]],
                [state[1][2][0],state[1][2][1],state[1][2][2],state[1][2][3],state[1][2][4],state[1][2][5],state[1][2][6],state[1][2][7]]
    ]]